In [1]:
!pip install googletrans==4.0.0-rc1
!pip install gTT
!pip install pydub
!pip install google-cloud-texttospeech

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=dc8de764d97c4d2d9c346d7f27158bbb7e8ba0357a9b443137896d3b74d95bb1
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [2]:
from googletrans import Translator

def detect_script_language(script):
    translator = Translator()
    try:
        detection = translator.detect(script)
        return detection.lang
    except Exception as e:
        return f"Error in language detection: {str(e)}"
script = "Hallo, wie geht es dir?"
print(detect_script_language(script))

de


In [3]:
import re
def split_script_by_personas(script, personas):

    # Escape and join persona names for regex
    personas_pattern = "|".join(re.escape(persona) for persona in personas)

    # Regex pattern to match persona lines
    pattern = rf"^\s*\**({personas_pattern})\**:\s*(.+)"

    lines = script.split("\n")
    dialogues = []
    current_persona = None
    current_dialogue = []

    for line in lines:
        match = re.match(pattern, line)
        if match:
            # Save the previous dialogue
            if current_persona and current_dialogue:
                dialogues.append((current_persona, " ".join(current_dialogue)))

            # Start a new dialogue
            current_persona = match.group(1)
            current_dialogue = [match.group(2)]
        elif current_persona:
            # Append continuation lines to the current dialogue
            current_dialogue.append(line.strip())

    # Add the last dialogue
    if current_persona and current_dialogue:
        dialogues.append((current_persona, " ".join(current_dialogue)))

    return [(persona, dialogue.replace('**', '').strip()) for persona, dialogue in dialogues]


In [4]:
from google.cloud import texttospeech
import os
import random
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

def generate_audio_google_cloud(text, language,i, file_path, no_of_people):

    # Map voice_id to Google Cloud language codes
    lang = {
    'en-GB': ['en-GB-Neural2-A','en-GB-Neural2-B','en-GB-Neural2-C','en-GB-Neural2-D'],
    'hi-IN': ['hi-IN-Neural2-A', 'hi-IN-Neural2-B','hi-IN-Neural2-C','hi-IN-Neural2-D'],
    'en-AU': ['en-AU-Neural2-A','en-AU-Neural2-B', 'en-AU-Neural2-C', 'en-AU-Neural2-D'],
    'fr-FR': ['fr-FR-Neural2-A', 'fr-FR-Neural2-B','fr-FR-Neural2-C','fr-FR-Neural2-D'],  # French
    'es-ES': ['es-ES-Neural2-A', 'es-ES-Neural2-B','es-ES-Neural2-C','es-ES-Neural2-D']  # Spanish
}
    lang_code = {
    'English': ['en-GB','en-AU'],
    'Hindi': ['hi-IN'],
    'French': ['fr-FR'],  # French
    'Spanish': ['es-ES']  # Spanish
}
    if language not in lang_code:
        raise ValueError(f"Unsupported voice_id: {language}. Supported options: {list(lang.keys())}")

    # Initialize the TTS client
    client = texttospeech.TextToSpeechClient()
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=1  # Set speaking rate
    )
    random_language_code = random.choice(lang_code[language])
    # Set the input text
    input_text = texttospeech.SynthesisInput(text=text)
    n = no_of_people
    # Set the voice parameters
    voice = texttospeech.VoiceSelectionParams(
        language_code=random_language_code,  # Use the selected language
         name=lang[random_language_code][i%n]
    )

    # Set the audio output configuration
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3  # Save as MP3
    )

    # Generate the audio
    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    # Save the audio to the specified file
    with open(file_path, "wb") as out:
        out.write(response.audio_content)

    return f"Audio file generated at {file_path}"


In [5]:
from pydub import AudioSegment
def create_podcast_audio(script, personas):
    try:
        lines = split_script_by_personas(script, personas)
        print(lines)
        audio_clips = []
        language_code = detect_script_language(script)
        print(language_code)
        m = {
              'en': 'English',
              'hi': 'Hindi',
              'fr': 'French',  # French
              'es': 'Spanish'  # Spanish
            }
        language = m[language_code]
        for i, (persona, dialogue) in enumerate(lines):
            output_path = f"voice_{i}.mp3"
            generate_audio_google_cloud(dialogue, language,i, output_path, 2)
            audio_clips.append(AudioSegment.from_file(output_path))

        final_audio = AudioSegment.empty()
        for clip in audio_clips:
            final_audio += clip

        final_audio.export("podcast_output.mp3", format="mp3")
        print("Podcast saved as podcast_output.mp3")
        return "podcast_output.mp3"
    except Exception as e:
        print(f"Audio generation error: {str(e)}")
        return None


In [6]:
script = """
Alice: नमस्ते और स्वागत है The AI Edge में! मैं हूं Alice।

Bob: और मैं हूं Bob। आज हम एक रोमांचक विषय पर चर्चा करेंगे: स्टॉक मार्केट पर आर्टिफिशियल इंटेलिजेंस का प्रभाव। AI हर इंडस्ट्री को बदल रहा है, और फाइनेंशियल वर्ल्ड भी इससे अछूता नहीं है।

Alice: सही कहा, Bob। हाल के सालों में स्टॉक मार्केट में AI का उपयोग काफी बढ़ गया है। अल्गोरिदमिक ट्रेडिंग से लेकर सेंटिमेंट एनालिसिस तक, AI यह तय करने के तरीके को पूरी तरह बदल रहा है कि निवेश कैसे किया जाए और पैसा कैसे प्रवाहित हो। चलिए शुरुआत करते हैं अल्गोरिदमिक ट्रेडिंग से।

Bob: बिल्कुल! अल्गोरिदमिक ट्रेडिंग या अल्गो-ट्रेडिंग, स्टॉक मार्केट में AI का सबसे बड़ा उपयोग है। AI-आधारित एल्गोरिदम पहले से तय मानदंडों जैसे प्राइस, वॉल्यूम, या जटिल पैटर्न के आधार पर ट्रेडिंग को तेज़ी से अंजाम देते हैं।

Alice: और इनकी गति अद्भुत होती है—ये एल्गोरिदम लाखों डेटा पॉइंट्स को सेकंड के भीतर एनालाइज़ करके ट्रेड्स कर सकते हैं। यह संस्थागत निवेशकों को रिटेल ट्रेडर्स के मुकाबले एक बड़ा फायदा देता है।

Bob: सही कहा, लेकिन इसका एक नकारात्मक पहलू भी है। ये हाई-फ्रीक्वेंसी ट्रेडिंग सिस्टम कभी-कभी मार्केट में अस्थिरता ला सकते हैं। 2010 के “फ्लैश क्रैश” को याद है?

Alice: बिल्कुल। उस घटना ने दिखाया कि बिना उचित निगरानी के एल्गोरिदम मार्केट वोलाटिलिटी को बढ़ा सकते हैं। यह याद दिलाता है कि AI पर पूरी तरह निर्भर नहीं हुआ जा सकता—इसे अभी भी मानव हस्तक्षेप की ज़रूरत है।

Alice: AI का एक और दिलचस्प उपयोग है सेंटिमेंट एनालिसिस। मशीन लर्निंग और नैचुरल लैंग्वेज प्रोसेसिंग का उपयोग करके AI न्यूज़ आर्टिकल्स, सोशल मीडिया और यहां तक कि अर्निंग्स कॉल ट्रांसक्रिप्ट्स का विश्लेषण कर सकता है ताकि मार्केट सेंटिमेंट को समझा जा सके।

Bob: यह वाकई कमाल है। सोचो, अगर AI ट्विटर पर ब्रेकिंग न्यूज़ या ट्रेंड्स को स्कैन कर सके। अगर Elon Musk टेस्ला के बारे में ट्वीट करते हैं, तो AI सिस्टम तुरंत उस सेंटिमेंट को एनालाइज़ करके इन्वेस्टमेंट स्ट्रैटेजी में बदलाव कर सकता है।

Alice: सही कहा! लेकिन जहां यह ट्रेडर्स को बढ़त देता है, वहीं यह कुछ नैतिक चिंताएं भी पैदा करता है। जैसे, अगर AI झूठी खबरों या पक्षपातपूर्ण जानकारी को बढ़ावा देने लगे तो क्या होगा?

Bob: यह एक महत्वपूर्ण मुद्दा है। चुनौती यह है कि इन सिस्टम्स को विश्वसनीय डेटा पर प्रशिक्षित किया जाए और उनके निर्णय लेने के तरीके पारदर्शी हों।
"""

personas = ["Bob", "Alice"]
create_podcast_audio(script, personas)

[('Alice', 'नमस्ते और स्वागत है The AI Edge में! मैं हूं Alice।'), ('Bob', 'और मैं हूं Bob। आज हम एक रोमांचक विषय पर चर्चा करेंगे: स्टॉक मार्केट पर आर्टिफिशियल इंटेलिजेंस का प्रभाव। AI हर इंडस्ट्री को बदल रहा है, और फाइनेंशियल वर्ल्ड भी इससे अछूता नहीं है।'), ('Alice', 'सही कहा, Bob। हाल के सालों में स्टॉक मार्केट में AI का उपयोग काफी बढ़ गया है। अल्गोरिदमिक ट्रेडिंग से लेकर सेंटिमेंट एनालिसिस तक, AI यह तय करने के तरीके को पूरी तरह बदल रहा है कि निवेश कैसे किया जाए और पैसा कैसे प्रवाहित हो। चलिए शुरुआत करते हैं अल्गोरिदमिक ट्रेडिंग से।'), ('Bob', 'बिल्कुल! अल्गोरिदमिक ट्रेडिंग या अल्गो-ट्रेडिंग, स्टॉक मार्केट में AI का सबसे बड़ा उपयोग है। AI-आधारित एल्गोरिदम पहले से तय मानदंडों जैसे प्राइस, वॉल्यूम, या जटिल पैटर्न के आधार पर ट्रेडिंग को तेज़ी से अंजाम देते हैं।'), ('Alice', 'और इनकी गति अद्भुत होती है—ये एल्गोरिदम लाखों डेटा पॉइंट्स को सेकंड के भीतर एनालाइज़ करके ट्रेड्स कर सकते हैं। यह संस्थागत निवेशकों को रिटेल ट्रेडर्स के मुकाबले एक बड़ा फायदा देता है।'), ('Bob', 'सही कहा, लेकिन इसक

'podcast_output.mp3'